In [2]:
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt
import numpy as np

In [3]:
airline_file = "Data/airlines.csv"
airline_df = pd.read_csv(airline_file)
airline_df.head()

FileNotFoundError: [Errno 2] File Data/airlines.csv does not exist: 'Data/airlines.csv'

In [ ]:
# Changing the names of columns and filtering columns we require
airline_clean_df = airline_df[["Airline ID", "Name", "IATA", "ICAO", "Country", "Active"]].copy()
airline_clean_df = airline_clean_df.rename(columns = {"Name": "Operator"})
airline_clean_df

In [ ]:
# Changing the name of Airlines in the Airlines.csv
airline_clean_df = airline_clean_df.replace({'Aeroflot Russian Airlines': 'Aeroflot',
                                            'Avianca - Aerovias Nacionales de Colombia': 'Avianca',
                                            'Private flight': 'Private'})
airline_clean_df

In [ ]:
airline_clean_df['Active'] = airline_clean_df['Active'].replace({'n': 'N'})
airline_clean_df

In [ ]:
# Checking if the Operator names have been changed
airline_clean_df.loc[airline_clean_df['Operator'] == 'Avianca']

In [ ]:
# Changing the names of Countries
airline_clean_df = airline_clean_df.replace({'ALASKA': 'United States',
                                                        'AVIANCA': 'Colombia',
                                                        'Russia]]': 'Russia'
                                                        })
airline_clean_df

In [ ]:
# Checking to see if the Country names have been changed
airline_clean_df.loc[airline_clean_df['Country'] == 'ALASKA']

In [ ]:
# # Resetin the index for airline_clean_df
# airline_clean_df = airline_clean_df.reset_index()
# # airline_clean_df = airline_df[["Airline ID", "Name", "Country", "Active"]]
# airline_clean_df

In [ ]:
airline_clean_df.loc[airline_clean_df['Operator'] == 'Aeroflot']

In [ ]:
# airline_clean_df['Operator'].drop_duplicates()

In [ ]:
# airline_clean_df['Operator'].value_counts().head(50)

In [ ]:
crash_file = "Data/Airline_crash_data.csv"
crash_df = pd.read_csv(crash_file)
crash_df.head()

In [ ]:
crash_clean_df = crash_df[["Date", "Year", "Location", "Operator", "Type", "Aboard", "Fatalities", "Summary"]].copy()
crash_clean_df["Date"] = pd.to_datetime(crash_clean_df["Date"])
crash_clean_df = crash_clean_df[crash_clean_df["Date"].dt.year >= 1970]

crash_clean_df = crash_clean_df.reset_index()

In [ ]:
crash_clean_df["Fatalities_rate"] = (crash_clean_df['Fatalities']/crash_clean_df['Aboard'])*100
crash_clean_df

In [ ]:
bins = [1970, 1979, 1989, 1999,2009]

group_names = ["70's", "80's", "90's", "00's"]
grouped_crash_clean_df = crash_clean_df.copy()
grouped_crash_clean_df['Year'] = pd.cut(crash_clean_df['Year'], bins, labels=group_names, include_lowest=True)
grouped_crash_clean_df.head()

In [ ]:
grouped_crash_clean_df['Date'].max()

In [ ]:
grouped_crash_clean_df['Date'].min()

In [ ]:
import statsmodels.api as sm
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix
import matplotlib.mlab as mlab
%matplotlib inline
import seaborn as sns

In [ ]:
from statsmodels.tools import add_constant as add_constant
grouped_crash_clean_constant = add_constant(grouped_crash_clean_df)
grouped_crash_clean_constant.head()

In [ ]:
grouped_crash_clean_constant.groupby('Year').count()

In [ ]:
grouped_crash_clean_constant.plot(x="Year",y="Fatalities",kind="line",figsize=(20,6))
plt.title('Count of Fatalities for different decades')
plt.xlabel('Decades')
plt.ylabel('Frequency')
plt.savefig('FatalityrateAcrossDecades.png')
plt.show()

In [ ]:
new1_df =grouped_crash_clean_constant.groupby('Year').mean()
new1_df

In [ ]:
new1_df.plot(y="Fatalities_rate",kind="line",figsize=(20,6))
plt.title('Fatality rate Frequency for different decades')
plt.xlabel('Decades')
plt.ylabel('Frequency')
plt.savefig('FatalityrateAcrossDecades.png')
plt.show()

In [ ]:
new2_df= grouped_crash_clean_df.copy()
new2_df.dropna(inplace=True)
new2_df.head()

In [ ]:
new3_df= new2_df["Location"].str.split(",",n=1, expand=True) 

new3_df.head(15)

In [ ]:
new2_df["City_crash"] =new3_df[0]
new2_df["Country_crash"] =new3_df[1]
new2_df

In [ ]:
new4_df= new2_df["Country_crash"].str.split(",",n=1, expand=True) 
new4_df

In [ ]:
import numpy as np

In [ ]:
new4_df[0] =np.where(~new4_df[1].isnull(),new4_df[1],new4_df[0])

In [ ]:
new4_df

In [ ]:
new2_df["Country_crash"] =new4_df[0]
new2_df

In [ ]:
countrydecade_df =new2_df[['Year','Aboard','Fatalities','Fatalities_rate','City_crash','Country_crash']].groupby(['Country_crash','Year']).count()
countrydecade_df.head(25)

In [ ]:
# # # All the crashes with two or more collisions, still trying a way to separating the two operators
# crash_clean_df[crash_clean_df['Operator'].str.contains('/', na=False)]

In [ ]:
# Attempting the resolve how to split the two collision rows
# crash_clean_df['Operator', 'Type'] = crash_clean_df['Operator', 'Type'].str.split("/") 
# crash_clean_df.explode('Operator', 'Type').reset_index(drop=False)


In [ ]:
# crash_clean_df = crash_clean_df.assign(Operator=crash_clean_df.Operator.str.split('/'))
# crash_clean_df.explode 

In [ ]:
crash_clean_df.head()

In [ ]:
# crash_clean_df.iloc[80:100]

In [ ]:
# Renaming Airlines so they match with the two datasets
crash_clean_df = crash_clean_df.replace({'United Air Lines': 'United Airlines',
                                         'AVIANCA': 'Avianca',
                                         'Varig': ' Varig Log',
                                         'Compania Dominicana de Aviacion': 'Dominicana de Aviaci',
                                         'Aviogenex (Yugoslavia)': 'Aviogenex',
                                         'Stirling Airways': 'Stirling Airlines',
                                         'Japan Air Lines': 'Japan Airlines',
                                         'Cathay PacifiAirways': 'Cathay Pacific',
                                         'British European Airways': 'British Airways',
                                         'Spantax': 'Spantax S.A.',
                                         'Eastern Air Lines': 'Balitmore Airways',
                                         'Alia Royal Jordanian Airlines': ' Royal Jordanian',
                                         'Libya Arab Airlines': ' Libyan Airlines',
                                         'Turkish Airlines(THY)': 'Turkish Airlines',
                                         'Martinair Holland NV': 'Martinair',
                                         'Ceskoslevenske Aerolinie': 'Czech Airlines',
                                         'Inex Adria Aviopromet': 'Adria Airways',
                                         'TAP (Air Portugal)': 'TAP Portugal',
                                         'Air India': 'Air India Limited',
                                         'Loftledidir IcelandiAirlines': 'Icelandair',
                                         'Iran National Airlines': 'Iran Air',
                                         'Dan Air Services': 'Dan-Air London',
                                         'Korean Airlines': ' Korean Air',
                                         'Arrow Airways': 'Arrow Air',
                                         'Mexicana': 'Mexicana de Aviaci',
                                         'Thai Airways': 'Thai Airways International',
                                         'Tajikistan Airlines': 'Tajikistan International Airlines',
                                         'Cebu PacifiAir': ' Cebu Pacific',
                                         'China Airlines (Taiwan)': 'China Airlines',
                                         'EgyptAir': 'Egyptair',
                                         'Scandinavian Airlines (SAS)': 'Scandinavian Airlines System',
                                         'Flash Air': 'Flash Airlines',
                                         'Subir (S7)': 'S7 Airlines',
                                         'Pulkovo Airlines': 'Rossiya-Russian Airlines',
                                         'TAM (Brazil)': 'TAM Brazilian Airlines',
                                         'One-Two-Go Airlines': 'One Two Go Airlines',
                                         'Philippine Air Lines': 'Philippine Airlines',
                                         'Garuda Indonesia Airlines': 'Garuda Indonesia',
                                         'LAN': 'LAN Airlines'})

In [ ]:
# Dropping the index column
crash_clean_df = crash_clean_df.drop(columns=['index'])
crash_clean_df

In [ ]:
# Merging the two tables together
airline_crash_df = pd.merge(airline_clean_df, crash_clean_df, how='outer', on='Operator')
airline_crash_df

In [ ]:
# Filling the NaN values with more meaningful data
airline_crash_df = airline_crash_df[airline_crash_df['ICAO'].notna()]
airline_crash_df = airline_crash_df[airline_crash_df['Airline ID'].notna()]
airline_crash_df = airline_crash_df[airline_crash_df['IATA'].notna()]
# airline_crash_df = airline_crash_df[airline_crash_df['Year'].notna()]
airline_crash_df[['Aboard', 'Fatalities']] = airline_crash_df[['Aboard', 'Fatalities']].fillna(0)
airline_crash_df[['Date', 'Location', 'Type', 'Summary']] = airline_crash_df[['Date', 'Location', 'Type', 'Summary']].fillna('No Inccident Recorded')
airline_crash_df.head(20)

In [ ]:
airline_crash_df

In [ ]:
# Test to see Airline values are entered 
airline_crash_df[airline_crash_df['Aboard'] >= 1]

In [ ]:
airline_crash_df[airline_crash_df['Active'] == 'Y']

In [ ]:
airline_crash_df[airline_crash_df['Date'] == 'No Inccident Recorded']

In [ ]:
# Number of crashes per Operator Country
airline_crash_df["Country"].value_counts().head(50)

In [ ]:
# Number of active operators
airline_crash_df["Active"].value_counts()

In [ ]:
# Operators with the most crashes
airline_crash_df["Operator"].value_counts().head(50)

In [ ]:
crash_clean_df["Fatalities_rate"] = (crash_clean_df['Fatalities']/crash_clean_df['Aboard'])*100
crash_clean_df

In [ ]:
bins = [1970, 1979, 1989, 1999,2009]

group_names = ["70's", "80's", "90's", "00's"]
grouped_crash_clean_df = crash_clean_df.copy()
grouped_crash_clean_df['Year'] = pd.cut(crash_clean_df['Year'], bins, labels=group_names, include_lowest=True)
grouped_crash_clean_df.head()

In [ ]:
grouped_crash_clean_df['Date'].max()

In [ ]:
grouped_crash_clean_df['Date'].min()

In [ ]:
import statsmodels.api as sm
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix
import matplotlib.mlab as mlab
%matplotlib inline
import seaborn as sns

In [ ]:
from statsmodels.tools import add_constant as add_constant
grouped_crash_clean_constant = add_constant(grouped_crash_clean_df)
grouped_crash_clean_constant.head()

In [ ]:
grouped_crash_clean_constant.groupby('Year').count()

In [ ]:
grouped_crash_clean_constant.plot(x="Year",y="Fatalities",kind="line",figsize=(20,6))
plt.title('Count of Fatalities for different decades')
plt.xlabel('Decades')
plt.ylabel('Frequency')
plt.savefig('FatalityrateAcrossDecades.png')
plt.show()

In [ ]:
new1_df =grouped_crash_clean_constant.groupby('Year').mean()
new1_df

In [ ]:
new1_df.plot(y="Fatalities_rate",kind="line",figsize=(20,6))
plt.title('Fatality rate Frequency for different decades')
plt.xlabel('Decades')
plt.ylabel('Frequency')
plt.savefig('FatalityrateAcrossDecades.png')
plt.show()

In [ ]:
new2_df= grouped_crash_clean_df.copy()
new2_df.dropna(inplace=True)
new2_df.head()

In [ ]:
new3_df= new2_df["Location"].str.split(",",n=1, expand=True) 

new3_df.head(15)

In [ ]:
new2_df["City_crash"] =new3_df[0]
new2_df["Country_crash"] =new3_df[1]
new2_df

In [ ]:
new4_df= new2_df["Country_crash"].str.split(",",n=1, expand=True) 
new4_df

In [ ]:
import numpy as np

In [ ]:
new4_df[0] =np.where(~new4_df[1].isnull(),new4_df[1],new4_df[0])

In [ ]:
new4_df

In [ ]:
new2_df["Country_crash"] =new4_df[0]
new2_df

In [ ]:
countrydecade_df =new2_df[['Year','Aboard','Fatalities','Fatalities_rate','City_crash','Country_crash']].groupby(['Country_crash','Year']).count()
countrydecade_df.head(25)

In [ ]:
maxcountry_crash_df = new2_df.value_counts(subset=['Country_crash'])
maxcountry_crash_df

In [ ]:
maxcountry_crash_df.columns=['Country_crash','count']
topten=maxcountry_crash_df.head(10)

In [ ]:
# maxcountry_crash_df.plot(x="Year",y="Fatalities",kind="line",figsize=(20,6))(y="Country_crash",y="Year",kind="line",figsize=(20,6))
topten.plot(kind="bar",figsize=(20,6))
plt.title('Countries with max crashes reported')
plt.xlabel('Crash site across Countries')
plt.ylabel('Frequency of occurence')
plt.savefig('FatalityrateAcrossCountries.png')
plt.show()
